In [2]:
%load_ext sql

In [3]:
%sql sqlite:///vivino.db


## QUERY 1 :  We want to highlight 10 wines to increase our sales. Which ones should we choose and why?
## I limited 10 wines that have the best ratings_count , I  did not order by rating_average because it depeneds on the number of counts 

In [3]:
%%sql
SELECT name, id , ratings_average, ratings_count
FROM wines 
ORDER BY ratings_count DESC
LIMIT 10 ; 

 * sqlite:///vivino.db
Done.


name,id,ratings_average,ratings_count
Cabernet Sauvignon,66284,4.6,157944
Brut Champagne,86684,4.6,146377
Tignanello,1652,4.5,142513
Sassicaia,5078,4.6,107646
60 Sessantanni Old Vines Primitivo di Manduria,11890,4.5,94289
Costasera Amarone della Valpolicella Classico,21929,4.3,93397
ARTEMIS Cabernet Sauvignon,1107336,4.4,81351
Amarone della Valpolicella Classico,7103,4.3,77515
Opus One,1911534,4.6,77053
Pauillac (Premier Grand Cru Classé),1684223,4.6,73615


## QUERY 2 :
## We have a limited marketing budget for this year. Which country should we prioritise and why?

In [4]:
%%sql
SELECT *
FROM countries
GROUP BY  name 
ORDER BY users_count DESC
LIMIT 4;

 * sqlite:///vivino.db
Done.


code,name,regions_count,users_count,wines_count,wineries_count
us,États-Unis,362,12273684,204060,28145
fr,France,1306,5973301,422503,67553
it,Italie,563,4270717,274658,42399
de,Allemagne,236,2549989,164533,13643


## QUERY 3 :
## We would like to give awards to the best wineries. Come up with 3 relevant ones. Which wineries should we choose and why?

In [5]:
%%sql
SELECT winery_id, wineries.name, (ratings_average) as avg_rating
FROM wines
JOIN wineries ON wineries.id = wines.winery_id
GROUP BY winery_id
ORDER BY avg_rating DESC
LIMIT 3;

 * sqlite:///vivino.db
Done.


winery_id,name,avg_rating
75712,Corte di Cama Sforzato di Valtellina,4.5
1651,Tenuta Tignanello 'Solaia',4.5
1652,Tignanello,4.4


## QUERY 4 : We detected that a big cluster of customers likes a specific combination of tastes. We identified a few keywords that match these tastes: coffee, toast, green apple, cream, and citrus . We would like you to find all the wines that are related to these keywords. Check that at least 10 users confirm those keywords, to ensure the accuracy of the selection. Additionally, identify an appropriate group name for this cluster.

In [6]:
%%sql
SELECT w.id, w.name, COUNT(*) as user_confirmed
FROM wines w
JOIN keywords_wine kw ON w.id = kw.wine_id
JOIN keywords k ON k.id = kw.keyword_id
WHERE k.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus') AND kw.count >= 10
GROUP BY w.id, w.name
HAVING COUNT(DISTINCT k.name) >= 5
ORDER BY user_confirmed DESC;

 * sqlite:///vivino.db
Done.


id,name,user_confirmed
18931,La Grande Année Brut Champagne,7
74304,Cristal Brut Champagne (Millésimé),7
79162,Belle Epoque Brut Champagne,7
79631,Vintage,7
79836,La Grande Dame Brut Champagne,7
86684,Brut Champagne,7
1105696,Trebbiano d'Abruzzo,7
1127349,Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne,7
1135215,Sauternes (Premier Grand Cru Classé),7
1136950,Comtes de Champagne Blanc de Blancs,7


## QUERY 5:
## We would like to select wines that are easy to find all over the world. Find the top 3 most common grapes all over the world and for each grape, give us the the 5 best rated wines.

In [7]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('vivino.db')
cur = conn.cursor()

# Query to find the top 3 most common grapes based on wines_count in most_used_grapes_per_country table
query5a = """
SELECT g.id, g.name, COUNT(m.wines_count) AS total_occurrences
FROM grapes g
JOIN most_used_grapes_per_country m ON g.id = m.grape_id
GROUP BY g.id, g.name
ORDER BY total_occurrences DESC
LIMIT 3;
"""

cur.execute(query5a)
top_grapes = cur.fetchall()

# Display the top 3 most common grapes based on wines_count in most_used_grapes_per_country table
if top_grapes:
    print("Top 3 Most Common Grapes All Over the World Based on Wines Count:")
    for i, grape in enumerate(top_grapes, 1):
        grape_id = grape[0]
        grape_name = grape[1]
        total_occurrences = grape[2]
        print(f"{i}. Grape ID: {grape_id}, Name: {grape_name}, Total Occurrences (Wines Count): {total_occurrences}")
else:
    print("No data found.")

# Close the cursor and connection
cur.close()
conn.close()

Top 3 Most Common Grapes All Over the World Based on Wines Count:
1. Grape ID: 2, Name: Cabernet Sauvignon, Total Occurrences (Wines Count): 12
2. Grape ID: 10, Name: Merlot, Total Occurrences (Wines Count): 11
3. Grape ID: 5, Name: Chardonnay, Total Occurrences (Wines Count): 6


## Q6 : create country leaderboard and a graph to show the average wine rating for each country***

In [8]:
%%sql
SELECT c.name, AVG(w.ratings_average) as avg_rating
FROM wines w
JOIN regions r ON w.region_id = r.id
JOIN countries c ON r.country_code = c.code
GROUP BY c.code
ORDER BY avg_rating DESC;

 * sqlite:///vivino.db
Done.


name,avg_rating
Israël,4.5
Allemagne,4.5
États-Unis,4.4905405405405405
Moldavie,4.4799999999999995
Hongrie,4.472727272727273
Afrique du Sud,4.45909090909091
Australie,4.458333333333333
France,4.447129909365559
Espagne,4.443617021276595
Portugal,4.435714285714286


## Q7 : One of our VIP clients likes Cabernet Sauvignon and would like our top 5 recommendations. Which wines would you recommend to him?

In [5]:
%%sql
SELECT id, name, ratings_average, ratings_count
FROM wines
WHERE name LIKE '%Cabernet Sauvignon%'
ORDER BY ratings_average DESC, ratings_count DESC
LIMIT 5;

 * sqlite:///vivino.db
Done.


id,name,ratings_average,ratings_count
1611255,Cabernet Sauvignon,4.8,2941
66294,Special Selection Cabernet Sauvignon,4.7,41236
66284,Cabernet Sauvignon,4.6,157944
3879,Cabernet Sauvignon (Signature),4.6,13730
2653222,CASK 23 Cabernet Sauvignon,4.6,9464
